In [3]:
import os
import threading
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression, LinearSVC, RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder, Tokenizer, StopWordsRemover, CountVectorizer, IDF, PCA, HashingTF


from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit, sum, when
from pyspark.sql.types import StringType

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [4]:
# Change file path
file_path = "/Users/hydraze/Library/CloudStorage/GoogleDrive-tohziyu2@gmail.com/My Drive/Studies/KU Leuven/Courses/Classes/Y1S2/Advanced Analytics in Business/Project/3/AdvancedAnalytics_Streaming-Text-Analytics/"
os.chdir(file_path)


In [5]:
# read pickled model via pipeline api
mPath =  file_path+"models/best_model"
best_model = PipelineModel.load(mPath)

In [6]:
# Toy predict function that returns a random probability. Normally you'd use your loaded globals()['my_model'] here
def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))

    # Convert to data frame
    df = spark.read.json(rdd)

    # Data cleaning (to update concurrently with the other jupyter notebook)
    
    # Extracting type of post: Show HN
    df = df.withColumn('isShowHN', when(df.title.contains("Show HN"), 1).otherwise(0))
    
    # Extracting time of day
    extract_time_of_day_udf = udf(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%H'))
    
    df = df.withColumn('time_of_day', extract_time_of_day_udf(df.posted_at))
    
    # Extracting day of week
    weekDay =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%w'))
    
    df = df.withColumn('day_of_week', weekDay(df.posted_at))

    # Fill null values
    df = df.na.fill({"title": "", "source_title": "", "source_text": ""})
    
    # And then predict using the loaded model (uncomment below):
    df_result = best_model.transform(df)
    df_result.select('aid', 'posted_at', 'comments', 'frontpage', 'prediction').show()

In [7]:
# Likely the usual streaming
ssc = StreamingContext(sc, 10)
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)
ssc_t = StreamingThread(ssc)
ssc_t.start()

24/05/24 22:07:26 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 22:07:26 WARN BlockManager: Block input-0-1716581246200 replicated to only 0 peer(s) instead of 1 peers


========= 2024-05-24 22:07:30 =========


24/05/24 22:07:30 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 22:07:30 WARN BlockManager: Block input-0-1716581250200 replicated to only 0 peer(s) instead of 1 peers


+--------+-------------------+--------+---------+----------+
|     aid|          posted_at|comments|frontpage|prediction|
+--------+-------------------+--------+---------+----------+
|40464076|2024-05-24 08:13:58|       0|    false|       0.0|
+--------+-------------------+--------+---------+----------+



24/05/24 22:07:34 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 22:07:34 WARN BlockManager: Block input-0-1716581254200 replicated to only 0 peer(s) instead of 1 peers
24/05/24 22:07:38 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 22:07:38 WARN BlockManager: Block input-0-1716581258200 replicated to only 0 peer(s) instead of 1 peers


...

24/05/24 23:53:40 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 23:53:40 WARN BlockManager: Block input-0-1716587619800 replicated to only 0 peer(s) instead of 1 peers


========= 2024-05-24 23:53:50 =========
+--------+-------------------+--------+---------+----------+
|     aid|          posted_at|comments|frontpage|prediction|
+--------+-------------------+--------+---------+----------+
|40467400|2024-05-24 15:52:53|       0|    false|       0.0|
+--------+-------------------+--------+---------+----------+



24/05/24 23:54:41 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 23:54:41 WARN BlockManager: Block input-0-1716587681000 replicated to only 0 peer(s) instead of 1 peers
24/05/24 23:54:42 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 23:54:42 WARN BlockManager: Block input-0-1716587682000 replicated to only 0 peer(s) instead of 1 peers
24/05/24 23:54:45 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 23:54:45 WARN BlockManager: Block input-0-1716587685000 replicated to only 0 peer(s) instead of 1 peers
24/05/24 23:54:49 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/24 23:54:49 WARN BlockManager: Block input-0-1716587688800 replicated to only 0 peer(s) instead of 1 peers


========= 2024-05-24 23:54:50 =========
+--------+-------------------+--------+---------+----------+
|     aid|          posted_at|comments|frontpage|prediction|
+--------+-------------------+--------+---------+----------+
|40467411|2024-05-24 15:54:04|       0|    false|       0.0|
|40467412|2024-05-24 15:54:07|       0|    false|       0.0|
|40467417|2024-05-24 15:54:22|       0|    false|       0.0|
|40467419|2024-05-24 15:54:39|       2|     true|       1.0|
+--------+-------------------+--------+---------+----------+



24/05/25 01:53:34 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 7076495 ms exceeds timeout 120000 ms
24/05/25 01:53:34 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/25 01:53:42 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [ ]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


24/05/17 15:41:16 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.base/java.net.SocketInputStream.socketRead0(Native Method)
	at java.base/java.net.SocketInputStream.socketRead(SocketInputStream.java:115)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:168)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at java.base/sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at java.base/sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.base/java.io.InputStreamReader.read(InputStreamReader.java:181)
	at java.base/java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:326)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:392)
	at org.apache.spark.streaming.dstream.SocketReceiver$$anon$2.getNext(SocketInputDStream.scala:121)
	at org.a